In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("elections_q.ipynb")

## Mini-lab - elections again

Here is a prerecorded walk-through video of the notebook. We encourage you to
reference this video as you work through the notebook. Run the cell below to
display the videos.

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo("hwRYs5ZRgW4", list = 'PLQCcNQgUcDfqSg049DVFZCQbupMY5Bn5Z', listType = 'playlist')

In this notebook, you are going to use several `pandas` methods. Reminder from
the lecture that you may press `shift+tab` on method parameters to see the
documentation for that method. For example, if you were using the `drop` method
in `pandas`, you could press `shift+tab` to see what `drop` is expecting.

In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

### **REVIEW:** `Groupby` and `Groupby` Shorthand

Let's now turn to use `groupby` from lectures 3 and 4.

### Elections

Let's start by reading in the election dataset from the `pandas` lectures.

In [ ]:
# Run this cell to load data from CSV file; no further action is needed.
elections = pd.read_csv("data/elections.csv")
elections.head(5)

As we saw before, we can `groupby` a specific column, e.g., `"Party"` and can print out the resulting sub-DataFrames. The output below can help you get an understanding of what `groupby` is actually doing.

An example is given below for elections since 1980.

In [ ]:
# Run this cell to print sub-DataFrames of a groupby object; no further action is needed.
for n, g in elections[elections["Year"] >= 1980].groupby("Party"):
    print(f"Name: {n}") # By the way, this is an "f string", a relatively new and great feature of Python
    display(g)

Recall that once we've formed groups, we can aggregate each sub-DataFrame (a.k.a. group) into a single row using an aggregation function. For example, if we use `.agg('mean')` on the groups above, we get back a single `DataFrame` where each group has been replaced by a single row. In each column for that aggregate row, the value that appears is the average of all values in that group.

For columns that are non-numeric, e.g., `"Result"`, the `pandas` version we're using (version 2.0.2) will error because we cannot compute the mean of the `Result` column. To remedy this, we add a `numeric_only=True` argument to our function calls so that we only calculate the `mean` for columns that contain numeric values. Alternatively, we can manually select only the numerical columns before calling the `agg` function so the aggregation is only applied to numerical columns.

In [ ]:
elections_after_1980 = elections[elections["Year"] >= 1980]

elections_after_1980.groupby("Party").agg('mean', numeric_only=True)

# alternatively, we can manually select only the numerical columns before calling `agg`
# elections_after_1980.groupby("Party")[['Year', 'Popular vote', '%']].agg('mean')

Equivalently we can use one of the shorthand aggregation functions, e.g. `.mean()`: 

In [ ]:
elections_after_1980.groupby("Party").mean(numeric_only=True)

Note that the index of the `DataFrame` returned by an `groupby.agg` call is no longer a set of numeric indices from $0$ to $N-1$. Instead, we see that the index for the example above is now the `Party`. If we want to restore our `DataFrame` so that `Party` is a column rather than the index, we can use `reset_index`.

In [ ]:
elections_after_1980.groupby("Party").mean(numeric_only=True).reset_index()

**IMPORTANT NOTE:** Notice that the code above consists of chained method calls. This sort of code is very common in `Pandas` programming and in data science in general. Such chained method calls can sometimes go many layers deep, so you might consider adding newlines between lines of code for clarity. For example, we could instead write the code above as:

In [ ]:
# pandas method chaining
(
elections.query("Year >= 1980").groupby("Party") 
                               .mean(numeric_only=True)  ## Computes the mean values by party
                               .reset_index()            ## Resets to a numerical index
)

Note that we have surrounded the entire call by a big set of parentheses so that `Python` doesn't complain about the indentation. An alternative is to use the \ symbol to indicate to `Python` that your code continues on to the next line!

In [ ]:
# pandas method chaining (alternative)
elections[elections["Year"] >= 1980].groupby("Party") \
                               .mean(numeric_only=True) \
                               .reset_index()     

**IMPORTANT NOTE:** You should NEVER solve problems like the one above using loops or list comprehensions. This is slow and also misses the entire point of this part of Data 100. 

Before we continue, we'll print out the election dataset again for your convenience. 

In [ ]:
elections.head(5)

### Question 1a

Using `groupby.agg` or one of the shorthand methods (`groupby.min`, `groupby.first`, etc.), create a `Series` object `best_result_percentage_only` that returns a `Series` showing the entire best result for every party, sorted in decreasing order. Your `Series` should include only parties that have earned at least 10% of the vote in some election. Your result should look like this:

<code>
Party
Democratic               61.344703
Republican               60.907806
Democratic-Republican    57.210122
National Union           54.951512
Whig                     53.051213
Liberal Republican       44.071406
National Republican      43.796073
Northern Democratic      29.522311
Progressive              27.457433
American                 21.554001
Independent              18.956298
Southern Democratic      18.138998
American Independent     13.571218
Constitutional Union     12.639283
Free Soil                10.138474
Name: %, dtype: float64
</code>
<br/>

A list of named `groupby.agg` shorthand methods are [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#aggregation) (you'll have to scroll down about one page).


In [ ]:
...
best_result_percentage_only = ...
# Show the result
best_result_percentage_only

In [ ]:
grader.check("q1a")


### Question 1b
Repeat Question 1a. However, this time, your result should be a `DataFrame` showing all available information (all columns) rather than only the percentage as a `Series`.

This question is a lot trickier than Question 1a. Make sure to check the
lecture slides if you're stuck! It's very easy to make a subtle mistake that
shows Woodrow Wilson and Howard Taft both winning the 2020 election.

For example, the first 3 rows of your table should be:

|Party | Year | Candidate      | Popular Vote | Result | %         |
|------|------|----------------|--------------|--------|-----------|
|**Democratic**  | 1964 | Lyndon Johnson | 43127041      | win   | 61.344703 |
|**Republican**  | 1972 | Richard Nixon | 47168710      | win   | 60.907806 |
|**Democratic-Republican**  | 1824 | Andrew Jackson | 151271      | loss   | 57.210122 |

Note that the index is `Party`. In other words, don't use `reset_index`.


In [ ]:
...
best_result = ...
# Show the result
best_result

In [ ]:
grader.check("q1b")

### **REVIEW:** `DataFrameGroupBy.filter`

Our `DataFrame` contains a number of parties that have never had a successful presidential run. For example, the 2020 elections included candidates from the Libertarian and Green parties, neither of which have elected a president.

In [ ]:
# Run this cell to print the last four rows; no further action is needed.
elections.tail(4)

Suppose we were conducting an analysis trying to focus our attention on parties that had elected a president. 

The most natural approach is to use `groupby.filter` [(docs)](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.filter.html). This is an incredibly powerful but subtle tool for filtering data.

The code below accomplishes the task at hand. It does this by creating a function that returns `True` if and only if a sub-`DataFrame` (a.k.a. group) contains at least one winner. This function, in turn, uses the `pandas` function `any` [(docs)](https://pandas.pydata.org/docs/reference/api/pandas.Series.any.html).

In [ ]:
# Run this cell to keep only the rows of parties that have 
# elected a president; no further action is needed.
def at_least_one_candidate_in_the_frame_has_won(frame):
    """Returns df with rows only kept for parties that have
    won at least one election
    """
    return (frame["Result"] == 'win').any()

winners_only = (
    elections
        .groupby("Party")
        .filter(at_least_one_candidate_in_the_frame_has_won)
)
winners_only.tail(5)

Alternately, we could have used a `lambda` function instead of explicitly defining a named function using `def`. 

In [ ]:
# Run this cell to keep only the rows of parties that have 
# elected a president; no further action is needed.
winners_only = (
    elections
        .groupby("Party")
        .filter(lambda x : (x["Result"] == "win").any())
)
winners_only.tail(5)


### Question 1c

Using `filter`, create a `DataFrame` object `major_party_results_since_1988` that includes all election results starting after 1988 (exclusive) but only shows a row if the Party it belongs to has earned at least 1% of the popular vote in ANY election since 1988.

For example, in 1988, you should not include the `New Alliance` candidate since this party has not earned 1% of the vote since 1988. However, you should include the `Libertarian` candidate from 1988 despite only having 0.47 percent of the vote in 1988, because in 2016 and 2020, the Libertarian candidates Gary Johnson and Jo Jorgensen exceeded 1% of the vote.

For example, the first three rows of the table you generate should look like:

|     |   Year | Candidate         | Party       |   Popular vote | Result   |         % |
|----:|-------:|:------------------|:------------|---------------:|:---------|----------:|
| 139 |   1992 | Andre Marrou      | Libertarian |       290087   | loss     | 0.278516  |
| 140 |   1992 | Bill Clinton      | Democratic  |       44909806 | win      | 43.118485 |
| 142 |   1992 | George H. W. Bush | Republican  |       39104550 | loss     |  37.544784|

*Hint*: The following questions might help you construct your solution. One of the lines should be identical to the `filter` examples shown above.

1) How can we **only** keep rows in the data starting after 1988 (exclusive)?
2) What column should we `groupby` to filter out parties that have earned at least 1% of the popular vote in ANY election since 1988?
3) How can we write an aggregation function that takes a sub-DataFrame and returns whether at least 1% of the vote has been earned in that sub-DataFrame? This may give you a hint about the second question!


In [ ]:
#- Maybe you want to define a function here?   Or use a lambda.
...
...
major_party_results_since_1988 = ...
# Show the first five rows
major_party_results_since_1988.head()

In [ ]:
grader.check("q1c")

### **REVIEW:** `str`

`Pandas` provides special purpose functions for working with specific common data types such as strings and dates, which you will learn about in more detail in Lecture 6. For example, the code below provides the length of every Candidate's name from our `elections` dataset. 

In [ ]:
elections["Candidate"].str.len()


### Question 2

Using `.str.split`, create a new `DataFrame` called `elections_with_first_name` with a new column `First Name` that is equal to the Candidate's first name.

See the `Pandas` `str`
[documentation](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.split.html)
for documentation on using `str.split`.

**Hint** — you might want to investigate the `expand` argument.


In [ ]:
# Make a copy of the data frame to work with.
elections_with_first_name = elections.copy()
...
# Show the result
elections_with_first_name

In [ ]:
grader.check("q2")

## Congratulations! You have finished this notebook




## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so you can check your answers.


In [ ]:
grader.check_all()